These sets of codes were made on 6/12/2025. The objective waas to compare the unsaturated hydraulic conductivity (K) values obtained using the Rosetta1 derived hydraulic parameters and the fitted parameters (K0 and L) with the Rosetta3 derived parameters. The Rosetta3 parameters were used to estimate the fitted parameters K0 and L using Rosetta1 run in Windows 98. Using the C2 model in Rosetta1 (old version), the Rosetta3 hydraulic parameters were manually added to generate K0 and L. The codes to generate the Rosetta3 parameters are in K_MesoSoilv2.0.ipynb.

In [ ]:
# Checking the system version used
import sys
print(sys.version)

In [ ]:
import os
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/GW_project/GWP/')

In [ ]:
##Installing rosetta

!pip install rosetta-soil
from rosetta import rosetta, SoilData

In [ ]:
import numpy as np
import pandas as pd
from scipy.io import loadmat    # handles matlab files

In [ ]:
# Loading Rosetta1 data
df1 = pd.read_csv('/content/drive/MyDrive/GW_project/GWP/MesoSoilv1_3.csv', header=0, skiprows=1)

# 60-cm depth data for all sites (every 6th row starting at index 5)
soil_properties = df1.iloc[5::6, [2, 3, 4, 5, 6, 7]].values.tolist()
id = df1.iloc[5::6, 0].astype(str).values  # Site IDs for Rosetta1

# Converting K0 and L from MesoSoilv1.3 to NumPy arrays
K0 = np.array([float(item) for sublist in df1.iloc[5::6, [13]].values.tolist() for item in sublist])
L = np.array([float(item) for sublist in df1.iloc[5::6, [14]].values.tolist() for item in sublist])

# Running Rosetta1
mean_1, stdev_1, codes_1 = rosetta(1, SoilData.from_array(soil_properties))

#Extracting Rosetta1 hydraulic parameters
theta_r1 = np.array([float(item[0]) for item in mean_1])
theta_s1 = np.array([float(item[1]) for item in mean_1])
alpha1 = np.array([(10**(float(item[2])) * 10.2) for item in mean_1])  # [1/kPa]
n1 = np.array([10**(float(item[3])) for item in mean_1])
m1 = np.array([1 - (1 / value) for value in n1])
Ks_1 = np.array([10**float(item[4]) for item in mean_1])

# Loading Rosetta3 data (with hydraulic parameters and other fitted parameters)
K0_df = pd.read_csv('/content/drive/MyDrive/GW_project/GWP/csv_outputs_mesoSoilv2.0/Rosetta3_with_K0_and_L.csv', header=1)

ID_added = K0_df.iloc[:, 0].astype(str).values
theta_r_added = K0_df.iloc[:, 1].astype(float).values
theta_s_added = K0_df.iloc[:, 2].astype(float).values
alpha_added = K0_df.iloc[:, 3].astype(float).values
n_added = K0_df.iloc[:, 4].astype(float).values
m_added = K0_df.iloc[:, 5].astype(float).values
Ks_added = K0_df.iloc[:, 6].astype(float).values
K0_added = K0_df.iloc[:, 7].astype(float).values
L_added = K0_df.iloc[:, 9].astype(float).values

In [ ]:
### Aligning the values by common Site IDs ready for pairwise comparison

common_ids, idx_v1, idx_v3 = np.intersect1d(id, ID_added, return_indices=True)

# Rosetta1
id_r1_aligned = id[idx_v1]
theta_r1_aligned = theta_r1[idx_v1]
theta_s1_aligned = theta_s1[idx_v1]
alpha1_aligned = alpha1[idx_v1]
n1_aligned = n1[idx_v1]
m1_aligned = m1[idx_v1]
Ks1_aligned = Ks_1[idx_v1]
K0_aligned = K0[idx_v1]
L_aligned = L[idx_v1]

# Rosetta3
id_r3_aligned = ID_added[idx_v3]
theta_r3_aligned = theta_r_added[idx_v3]
theta_s3_aligned = theta_s_added[idx_v3]
alpha3_aligned = alpha_added[idx_v3]
n3_aligned = n_added[idx_v3]
m3_aligned = m_added[idx_v3]
Ks3_aligned = Ks_added[idx_v3]
K0_added_aligned = K0_added[idx_v3]
L_added_aligned = L_added[idx_v3]

"""
# Printing of results
print("Rosetta1 aligned theta_r values with Site IDs:")
for site_id, value in zip(common_ids, theta_r1_aligned):
    print(f"{site_id}: {value:.5f}")

print("Rosetta3 aligned theta_r values with Site IDs:")
for site_id, value in zip(common_ids, theta_r3_aligned):
    print(f"{site_id}: {value:.5f}")
"""

'\n# Printing of results\nprint("Rosetta1 aligned theta_r values with Site IDs:")\nfor site_id, value in zip(common_ids, theta_r1_aligned):\n    print(f"{site_id}: {value:.5f}")\n\nprint("Rosetta3 aligned theta_r values with Site IDs:")\nfor site_id, value in zip(common_ids, theta_r3_aligned):\n    print(f"{site_id}: {value:.5f}")\n'

In [ ]:
### Rosetta1 vs Rosetta3 unsaturated hydraulic conductivity (K)
import datetime

np.seterr(over='raise', invalid='raise')  # Enable overflow exceptions

# Dictionaries to store results
df = {}
tr60_data = {}
time = {}
matric_potential = {}
vwc_r1 = {}
vwc_r3 = {}
Se_r1 = {}
Se_r3 = {}
K_r1 = {}
K_r3 = {}

# Constants for Zhang et al. (2019)
mp_min = -2083
a = 3.35
deltaT_inf = 3.17

# Creating a lookup from site_id to aligned index
aligned_index = {common_ids[i]: i for i in range(len(common_ids))}

# Directory containing all the .mat files
directory = '/content/drive/MyDrive/GW_project/GWP/deltaT_mat_mesoSoilv1.3'

# Getting all .mat files in the directory
file_names = sorted([f for f in os.listdir(directory) if f.endswith('.mat')])

# Loading each file into a DataFrame and storing it with its site_id as the key
for file_name in file_names:
    full_key = os.path.splitext(file_name)[0]
    site_id = full_key.replace("deltaT", "")

    # Skipping if site is not in aligned_index to avoid mismatch
    if site_id not in aligned_index:
        continue

    idx = aligned_index[site_id]

    file_path = os.path.join(directory, file_name)
    data = loadmat(file_path)
    df[site_id] = pd.DataFrame(data[full_key])

    # Extracting tr60 and time while skipping NaNs
    tr60_raw = df[site_id].iloc[2:, 4].astype(float)
    time_raw = df[site_id].iloc[2:, 0].astype(float)

    valid_mask = ~np.isnan(tr60_raw)
    tr60 = tr60_raw[valid_mask].values
    time_vals = time_raw[valid_mask].values

    if len(tr60) == 0:
        continue

    tr60_data[site_id] = tr60
    time[site_id] = pd.to_datetime([datetime.datetime.fromordinal(int(x)) + datetime.timedelta(days=x%1) - datetime.timedelta(days=366) for x in time_vals])

    try:
        # Matric potential vectorized
        mp = mp_min / (1 + np.exp(-a * (tr60 - deltaT_inf)))
        matric_potential[site_id] = mp

        # Rosetta1 params at idx
        tr1 = theta_r1_aligned[idx]
        ts1 = theta_s1_aligned[idx]
        a1 = alpha1_aligned[idx]
        n1v = n1_aligned[idx]
        m1v = m1_aligned[idx]
        K01 = K0_aligned[idx]
        L1 = L_aligned[idx]
        Ks1_val = Ks1_aligned[idx]

        # Rosetta3 params at idx
        tr3 = theta_r3_aligned[idx]
        ts3 = theta_s3_aligned[idx]
        a3 = alpha3_aligned[idx]
        n3v = n3_aligned[idx]
        m3v = m3_aligned[idx]
        K03 = K0_added_aligned[idx]
        L3 = L_added_aligned[idx]
        Ks3_val = Ks3_aligned[idx]

        # Volumetric water content (vwc) Rosetta1 [cm3/cm3]
        base1 = 1 + (-a1 * mp)**n1v
        vwc1 = tr1 + (ts1 - tr1) * base1**(-m1v)
        vwc_r1[site_id] = vwc1

        # Volumetric water content (vwc) Rosetta3 [cm3/cm3]
        base3 = 1 + (-a3 * mp)**n3v
        vwc3 = tr3 + (ts3 - tr3) * base3**(-m3v)
        vwc_r3[site_id] = vwc3

        # Effective saturation (Se) Rosetta1 [Zhang et al., 2019]
        Se1 = (vwc1 - tr1) / (ts1 - tr1)
        Se_r1[site_id] = Se1

        # Effective saturation (Se) Rosetta3 [Zhang et al., 2019]
        Se3 = (vwc3 - tr3) / (ts3 - tr3)
        Se_r3[site_id] = Se3

        # Unsaturated hydraulic conductivity using Rosetta1 parameters [cm/day]
        term1 = (1 - Se1**(1/m1v))
        K_r1_vals = K01 * Se1**L1 * (1 - term1**m1v)**2
        K_r1[site_id] = K_r1_vals

        # Unsaturated hydraulic conductivity using Rosetta3 parameters [cm/day]
        term3 = (1 - Se3**(1/m3v))
        K_r3_vals = K03 * Se3**L3 * (1 - term3**m3v)**2
        K_r3[site_id] = K_r3_vals

    except FloatingPointError as e:
        print(f"Floating point error at site {site_id}: {e}")
        continue

"""
# Checking for the vwc and K results
# Printing the first 10 vwc1 values for each site
for site_id, vwc_vals in vwc_r1.items():
    print(f"Site {site_id} - first 10 VWC_R1 values:\n{vwc_vals[:10]}\n")

# Printing the  first 10 K1 values for each site
for site_id, K_r1_vals in K_r1.items():
    print(f"Site {site_id} - first 10 K_r1 values:\n{K_r1_vals[:10]}\n")
"""

'\n# Checking for the vwc and K results\n# Printing the first 10 vwc1 values for each site\nfor site_id, vwc_vals in vwc_r1.items():\n    print(f"Site {site_id} - first 10 VWC_R1 values:\n{vwc_vals[:10]}\n")\n\n# Printing the  first 10 K1 values for each site\nfor site_id, K_r1_vals in K_r1.items():\n    print(f"Site {site_id} - first 10 K_r1 values:\n{K_r1_vals[:10]}\n")\n'

In [ ]:
### Calculating the daily average volumetric water content [cm3/cm3]

# Dictionaries to store daily average vwc
vwc_r1_ave = {}
vwc_r3_ave = {}

# Average vwc_r1
for site_id, values in vwc_r1.items():
    vwc_values = [value for value in values if not np.isnan(value)]
    site_key = site_id[:4]                 # keep consistent with formatting
    if vwc_values:
        vwc_r1_ave[site_key] = sum(vwc_values) / len(vwc_values)
    else:
        vwc_r1_ave[site_key] = float('nan')

# Average vwc_r3
for site_id, values in vwc_r3.items():
    vwc_values = [value for value in values if not np.isnan(value)]
    site_key = site_id[:4]
    if vwc_values:
        vwc_r3_ave[site_key] = sum(vwc_values) / len(vwc_values)
    else:
        vwc_r3_ave[site_key] = float('nan')



# Quick checking of the results
for site_id, avg in vwc_r1_ave.items():
    print(f'vwc_r1_ave {site_id}: {avg:.5f}')

print('-----------------------------')

for site_id, avg in vwc_r3_ave.items():
    print(f'vwc_r3_ave {site_id}: {avg:.5f}')


vwc_r1_ave ACME: 0.19748
vwc_r1_ave ALTU: 0.25839
vwc_r1_ave ANT2: 0.24999
vwc_r1_ave ANTL: 0.17183
vwc_r1_ave APAC: 0.23359
vwc_r1_ave ARD2: 0.30364
vwc_r1_ave ARNE: 0.14088
vwc_r1_ave BEAV: 0.13593
vwc_r1_ave BIXB: 0.24879
vwc_r1_ave BLAC: 0.28384
vwc_r1_ave BOIS: 0.23647
vwc_r1_ave BOWL: 0.26936
vwc_r1_ave BREC: 0.31210
vwc_r1_ave BRIS: 0.26740
vwc_r1_ave BUFF: 0.18783
vwc_r1_ave BURN: 0.11357
vwc_r1_ave BUTL: 0.25553
vwc_r1_ave BYAR: 0.22104
vwc_r1_ave CARL: 0.24515
vwc_r1_ave CENT: 0.29010
vwc_r1_ave CHAN: nan
vwc_r1_ave CHER: 0.12441
vwc_r1_ave CHEY: 0.19208
vwc_r1_ave COPA: 0.29142
vwc_r1_ave DURA: 0.36641
vwc_r1_ave ELKC: 0.18015
vwc_r1_ave ELRE: 0.28054
vwc_r1_ave ERIC: 0.14805
vwc_r1_ave EUFA: 0.23302
vwc_r1_ave FAIR: 0.23541
vwc_r1_ave FITT: 0.30506
vwc_r1_ave FORA: 0.29656
vwc_r1_ave FREE: 0.13557
vwc_r1_ave FTCB: 0.16031
vwc_r1_ave GOOD: 0.13265
vwc_r1_ave GRA2: 0.25606
vwc_r1_ave HASK: 0.31195
vwc_r1_ave HECT: 0.18024
vwc_r1_ave HINT: 0.11212
vwc_r1_ave HOBA: 0.27168
vwc_

In [ ]:
### Calculating the grand mean and sample standard deviation for the volumetric water content (cm3/cm3)

def mean_std(data_dict):
  values = [value for value in data_dict.values() if not np.isnan(value)]
  if values:
    mean = round(sum(values) / len(values), 3)
    std = round(np.std(values, ddof=1), 3)
    return mean, std
  else:
    return float('nan'), float('nan')

grand_mean_vwc_r1, std_vwc_r1 = mean_std(vwc_r1_ave)
grand_mean_vwc_r3, std_vwc_r3 = mean_std(vwc_r3_ave)


print(f'grand_mean_vwc_r1: {grand_mean_vwc_r1}')
print(f'grand_mean_vwc_r3: {grand_mean_vwc_r3}')

grand_mean_vwc_r1: 0.247
grand_mean_vwc_r3: 0.264


In [ ]:
### Calculating the annual average unsaturated hydraulic conductivity [mm/year]

# Dictionaries to store daily average K
K_r1_ave = {}
K_r3_ave = {}

# Average K_r1
for site_id, values in K_r1.items():
    K_values = [value for value in values if not np.isnan(value)]
    site_key = site_id[:4]               # keep consistent with formatting
    if K_values:
        K_r1_ave[site_key] = sum(K_values) / len(K_values)
        K_r1_ave[site_key] *= 3650       # converting cm/day to mm/yr
    else:
        K_r1_ave[site_key] = float('nan')

# Average K_r3
for site_id, values in K_r3.items():
    K_values = [value for value in values if not np.isnan(value)]
    site_key = site_id[:4]
    if K_values:
        K_r3_ave[site_key] = sum(K_values) / len(K_values)
        K_r3_ave[site_key] *= 3650      # converting cm/day to mm/yr
    else:
        K_r3_ave[site_key] = float('nan')


# Quick checking of the results
print("Rosetta1 Average K values:")
for i, (site_id, avg) in enumerate(K_r1_ave.items(), start=1):
    print(f'{i}. K_r1_ave {site_id}: {avg:.5f}')

print('-----------------------------')
# min/max values of K for Rosetta1
print(f"\nRosetta1 - Min K: {min(K_r1_ave.values()):.5f} (Site: {min(K_r1_ave, key=K_r1_ave.get)})")
print(f"Rosetta1 - Max K: {max(K_r1_ave.values()):.5f} (Site: {max(K_r1_ave, key=K_r1_ave.get)})")

print('-----------------------------')

print("\nRosetta3 Average K values:")
for i, (site_id, avg) in enumerate(K_r3_ave.items(), start=1):
    print(f'{i}. K_r3_ave {site_id}: {avg:.5f}')

print('-----------------------------')
# min/max values of K for Rosetta3
print(f"\nRosetta3 - Min K: {min(K_r3_ave.values()):.5f} (Site: {min(K_r3_ave, key=K_r3_ave.get)})")
print(f"Rosetta3 - Max K: {max(K_r3_ave.values()):.5f} (Site: {max(K_r3_ave, key=K_r3_ave.get)})")

Rosetta1 Average K values:
1. K_r1_ave ACME: 161.84110
2. K_r1_ave ALTU: 12.81344
3. K_r1_ave ANT2: 361.58903
4. K_r1_ave ANTL: 404.89602
5. K_r1_ave APAC: 95.55843
6. K_r1_ave ARD2: 63.67677
7. K_r1_ave ARNE: 12.66534
8. K_r1_ave BEAV: 15.76959
9. K_r1_ave BIXB: 537.49492
10. K_r1_ave BLAC: 132.89576
11. K_r1_ave BOIS: 9.66827
12. K_r1_ave BOWL: 143.14436
13. K_r1_ave BREC: 44.04690
14. K_r1_ave BRIS: 606.12324
15. K_r1_ave BUFF: 30.07374
16. K_r1_ave BURN: 115.63343
17. K_r1_ave BUTL: 39.27385
18. K_r1_ave BYAR: 130.15007
19. K_r1_ave CARL: 117.88733
20. K_r1_ave CENT: 358.38074
21. K_r1_ave CHAN: nan
22. K_r1_ave CHER: 7.44639
23. K_r1_ave CHEY: 45.13794
24. K_r1_ave COPA: 458.80265
25. K_r1_ave DURA: 76.00959
26. K_r1_ave ELKC: 139.16465
27. K_r1_ave ELRE: 111.17741
28. K_r1_ave ERIC: 28.86600
29. K_r1_ave EUFA: 319.66145
30. K_r1_ave FAIR: 99.76766
31. K_r1_ave FITT: 136.92510
32. K_r1_ave FORA: 578.12883
33. K_r1_ave FREE: 20.65121
34. K_r1_ave FTCB: 88.13477
35. K_r1_ave GOOD: 1

In [ ]:
### Calculating the grand mean and sample standard deviation for unsaturated hydraulic conductivity (mm/yr)
### number of Mesonet sites included = 90

def mean_std(data_dict):
  values = [value for value in data_dict.values() if not np.isnan(value)]
  if values:
    mean = round(sum(values) / len(values), 3)
    std = round(np.std(values, ddof=1), 3)
    return mean, std
  else:
    return float('nan'), float('nan')

grand_mean_K_r1, std_K_r1 = mean_std(K_r1_ave)
grand_mean_K_r3, std_K_r3 = mean_std(K_r3_ave)

print(f'grand_mean_K_r1: {grand_mean_K_r1}')
print(f'grand_mean_K_r3: {grand_mean_K_r3}')

grand_mean_K_r1: 184.349
grand_mean_K_r3: 116.587


In [ ]:
### Saving the average results for vwc and K for each site in a csv file

import csv

# Defining the file path
file_path = '/content/drive/MyDrive/GW_project/GWP/csv_outputs_Rosetta1_Rosetta3_comparison/Rosetta1_vs_Rosetta3_vwc_K.csv'

# Get a union of all keys (site IDs)
all_site_ids = sorted(set(vwc_r1_ave.keys()).union(vwc_r3_ave.keys(), K_r1_ave.keys(), K_r3_ave.keys()))

# Saving into csv file
with open(file_path, mode='w', newline='') as file:
  writer = csv.writer(file)

  # Writing the header
  writer.writerow(['SITE_ID', 'Average vwc_Ros1', 'Average vwc_Ros3', 'Average K_Ros1', 'Average K_Ros3'])

  # Writing the data rows
  for site_id in all_site_ids:
    vwc1 = vwc_r1_ave.get(site_id, 'nan')
    vwc3 = vwc_r3_ave.get(site_id, 'nan')
    k_ros1 = K_r1_ave.get(site_id, 'nan')
    k_ros3 = K_r3_ave.get(site_id, 'nan')

    writer.writerow([site_id, vwc1, vwc3, k_ros1, k_ros3])

print("CSV file saved successfully.")

CSV file saved successfully.


In [ ]:
### Saving the average vwc, K, and the hydraulic parameters with K0 and L in a csv file

# Loading the Rosetta1_vs_Rosetta3_vwc_K.csv
data_f = pd.read_csv('/content/drive/MyDrive/GW_project/GWP/csv_outputs_Rosetta1_Rosetta3_comparison/Rosetta1_vs_Rosetta3_vwc_K.csv', header=0)

# Extracting SITE_IDs to use as filter
site_ids_90 = data_f['SITE_ID'].values

# Finding index positions of those 90 sites in the aligned arrays (id_r1_aligned and id_r3_aligned)
site_indices_r1 = [np.where(id_r1_aligned == site)[0][0] for site in site_ids_90]
site_indices_r3 = [np.where(id_r3_aligned == site)[0][0] for site in site_ids_90]

# Subset parameters for those 90 sites
# Rosetta1
ros1_dict = {
    'SITE_ID': id_r1_aligned[site_indices_r1],
    'theta_r1': theta_r1_aligned[site_indices_r1],
    'theta_s1': theta_s1_aligned[site_indices_r1],
    'alpha1': alpha1_aligned[site_indices_r1],
    'n1': n1_aligned[site_indices_r1],
    'm1': m1_aligned[site_indices_r1],
    'Ks1': Ks1_aligned[site_indices_r1],
    'K0': K0_aligned[site_indices_r1],
    'L': L_aligned[site_indices_r1],
    'Average vwc_Ros1': data_f['Average vwc_Ros1'].values,
    'Average K_Ros1': data_f['Average K_Ros1'].values}

rosetta1_df = pd.DataFrame(ros1_dict)

# Rosetta3
ros3_dict = {
    'SITE_ID': id_r3_aligned[site_indices_r3],
    'theta_r3': theta_r3_aligned[site_indices_r3],
    'theta_s3': theta_s3_aligned[site_indices_r3],
    'alpha3': alpha3_aligned[site_indices_r3],
    'n3': n3_aligned[site_indices_r3],
    'm3': m3_aligned[site_indices_r3],
    'Ks3': Ks3_aligned[site_indices_r3],
    'K0': K0_added_aligned[site_indices_r3],
    'L': L_added_aligned[site_indices_r3],
    'Average vwc_Ros3': data_f['Average vwc_Ros3'].values,
    'Average K_Ros3': data_f['Average K_Ros3'].values}

rosetta3_df = pd.DataFrame(ros3_dict)

# Rounding of values
rosetta1_df, rosetta3_df = pd.DataFrame(ros1_dict).round(3), pd.DataFrame(ros3_dict).round(3)

# Saving to  a csv file
rosetta1_df.to_csv('/content/drive/MyDrive/GW_project/GWP/csv_outputs_Rosetta1_Rosetta3_comparison/Rosetta1_parameters.csv', index=False)
rosetta3_df.to_csv('/content/drive/MyDrive/GW_project/GWP/csv_outputs_Rosetta1_Rosetta3_comparison/Rosetta3_parameters.csv', index=False)

In [ ]:
# Average values of water retention parameters for all OK Mesonet sites at the 60-cm depth

# Rosetta1 parameters
theta_r1_array = np.array(theta_r1_aligned)
theta_s1_array = np.array(theta_s1_aligned)
alpha1_array = np.array(alpha1_aligned)
n1_array = np.array(n1_aligned)
Ks1_array = np.array(Ks1_aligned)
K0_array = np.array(K0_aligned)
L_array = np.array(L_aligned)

#Rosetta3 parameters
theta_r3_array = np.array(theta_r3_aligned)
theta_s3_array = np.array(theta_s3_aligned)
alpha3_array = np.array(alpha3_aligned)
n3_array = np.array(n3_aligned)
Ks3_array = np.array(Ks3_aligned)
K0_added_array = np.array(K0_added_aligned)
L_added_array = np.array(L_added_aligned)

# Calculating the average and sample standard deviation while skipping nan values
theta_r1_Ave = round(np.nanmean(theta_r1_array), 3)
theta_r1_Std = round(np.nanstd(theta_r1_array, ddof=1), 3)
theta_s1_Ave = round(np.nanmean(theta_s1_array), 3)
theta_s1_Std = round(np.nanstd(theta_s1_array, ddof=1), 3)
alpha1_Ave = round(np.nanmean(alpha1_array), 3)
alpha1_Std = round(np.nanstd(alpha1_array, ddof=1), 3)
n1_Ave = round(np.nanmean(n1_array), 3)
n1_Std = round(np.nanstd(n1_array, ddof=1), 3)
Ks1_Ave = round(np.nanmean(Ks1_array), 3)
Ks1_Std = round(np.nanstd(Ks1_array, ddof=1), 3)
K0_Ave = round(np.nanmean(K0_array), 3)
K0_Std = round(np.nanstd(K0_array, ddof=1), 3)
L_Ave = round(np.nanmean(L_array), 3)
L_Std = round(np.nanstd(L_array, ddof=1), 3)

theta_r3_Ave = round(np.nanmean(theta_r3_array), 3)
theta_r3_Std = round(np.nanstd(theta_r3_array, ddof=1), 3)
theta_s3_Ave = round(np.nanmean(theta_s3_array), 3)
theta_s3_Std = round(np.nanstd(theta_s3_array, ddof=1), 3)
alpha3_Ave = round(np.nanmean(alpha3_array), 3)
alpha3_Std = round(np.nanstd(alpha3_array, ddof=1), 3)
n3_Ave = round(np.nanmean(n3_array), 3)
n3_Std = round(np.nanstd(n3_array, ddof=1), 3)
Ks3_Ave = round(np.nanmean(Ks3_array), 3)
Ks3_Std = round(np.nanstd(Ks3_array, ddof=1), 3)
K0_added_Ave = round(np.nanmean(K0_added_array), 3)
K0_added_Std = round(np.nanstd(K0_added_array, ddof=1), 3)
L_added_Ave = round(np.nanmean(L_added_array), 3)
L_added_Std = round(np.nanstd(L_added_array, ddof=1), 3)


In [ ]:
### Printing the Rosetta1 hydraulic and fitted parameters and the vwc for each site

print("Site ID | theta_r | theta_s | alpha | n | m | Ks | K0 | L")
print("----------------------------------------------------------------------------")
for i, site_id in enumerate(common_ids):
    print(f"{site_id:7} | "
          f"{theta_r1_aligned[i]:.3f} | "
          f"{theta_s1_aligned[i]:.3f} | "
          f"{alpha1_aligned[i]:.3f} | "
          f"{n1_aligned[i]:.3f} | "
          f"{m1_aligned[i]:.3f} | "
          f"{Ks1_aligned[i]:.3f} | "
          f"{K0_aligned[i]:.3f} | "
          f"{L_aligned[i]:.3f}")

Site ID | theta_r | theta_s | alpha | n | m | Ks | K0 | L
----------------------------------------------------------------------------
ACME    | 0.055 | 0.365 | 0.343 | 1.319 | 0.242 | 22.202 | 12.800 | -1.710
ADAX    | 0.043 | 0.331 | 0.090 | 1.239 | 0.193 | 0.822 | 2.700 | -1.060
ALTU    | 0.064 | 0.396 | 0.130 | 1.186 | 0.157 | 1.259 | 3.100 | -2.280
ALV2    | 0.074 | 0.398 | 0.228 | 1.198 | 0.166 | 6.766 | 5.400 | -2.880
ANT2    | 0.047 | 0.348 | 0.209 | 1.292 | 0.226 | 10.224 | 7.200 | -1.420
ANTL    | 0.015 | 0.271 | 0.288 | 1.361 | 0.265 | 26.934 | 17.900 | -1.170
APAC    | 0.057 | 0.388 | 0.119 | 1.308 | 0.235 | 6.678 | 3.600 | -0.740
ARD2    | 0.071 | 0.416 | 0.164 | 1.175 | 0.149 | 3.277 | 3.700 | -2.900
ARNE    | 0.055 | 0.406 | 0.332 | 1.359 | 0.264 | 36.188 | 15.600 | -1.340
BEAV    | 0.079 | 0.407 | 0.481 | 1.446 | 0.309 | 29.544 | 20.400 | -1.570
BESS    | nan | nan | nan | nan | nan | nan | -9.900 | -9.900
BIXB    | 0.031 | 0.357 | 0.082 | 1.460 | 0.315 | 21.324 | 3.500

In [ ]:
### Printing the Rosetta3 hydraulic and the fitted parameters for each site

print("Site ID | theta_r | theta_s | alpha | n | m | Ks | K0 | L")
print("----------------------------------------------------------------------------")
for i, site_id in enumerate(common_ids):
    print(f"{site_id:7} | "
          f"{theta_r3_aligned[i]:.3f} | "
          f"{theta_s3_aligned[i]:.3f} | "
          f"{alpha3_aligned[i]:.3f} | "
          f"{n3_aligned[i]:.3f} | "
          f"{m3_aligned[i]:.3f} | "
          f"{Ks3_aligned[i]:.3f} | "
          f"{K0_added_aligned[i]:.3f} | "
          f"{L_added_aligned[i]:.3f}")

Site ID | theta_r | theta_s | alpha | n | m | Ks | K0 | L
----------------------------------------------------------------------------
ACME    | 0.068 | 0.361 | 0.193 | 1.342 | 0.255 | 20.577 | 1.986 | -2.888
ADAX    | 0.080 | 0.318 | 0.061 | 1.212 | 0.175 | 0.988 | 1.059 | -4.215
ALTU    | 0.118 | 0.372 | 0.073 | 1.173 | 0.147 | 1.137 | 0.949 | -6.149
ALV2    | 0.115 | 0.382 | 0.130 | 1.204 | 0.170 | 4.051 | 1.369 | -5.561
ANT2    | 0.066 | 0.343 | 0.133 | 1.299 | 0.230 | 8.752 | 1.706 | -3.134
ANTL    | 0.027 | 0.275 | 0.198 | 1.384 | 0.277 | 22.683 | 2.155 | -2.086
APAC    | 0.083 | 0.377 | 0.086 | 1.287 | 0.223 | 6.486 | 1.442 | -2.998
ARD2    | 0.126 | 0.395 | 0.098 | 1.164 | 0.141 | 2.006 | 1.094 | -7.191
ARNE    | 0.064 | 0.394 | 0.194 | 1.367 | 0.269 | 44.540 | 2.133 | -2.605
BEAV    | 0.082 | 0.396 | 0.246 | 1.467 | 0.319 | 51.289 | 2.271 | -2.290
BESS    | nan | nan | nan | nan | nan | nan | nan | nan
BIXB    | 0.051 | 0.365 | 0.070 | 1.437 | 0.304 | 17.569 | 1.606 | -1.655
B

In [ ]:
### Printing the average values of water retention parameters for all OK Mesonet sites at the 60-cm depth

print(f'theta_r1_Ave: {theta_r1_Ave}')
print(f'theta_s1_Ave: {theta_s1_Ave}')
print(f'alpha1_Ave: {alpha1_Ave}')
print(f'n1_Ave: {n1_Ave}')
print(f'Ks1_Ave: {Ks1_Ave}')
print(f'K0_Ave: {K0_Ave}')
print(f'L_Ave: {L_Ave}')
print('---------------------------')
print(f'theta_r3_Ave: {theta_r3_Ave}')
print(f'theta_s3_Ave: {theta_s3_Ave}')
print(f'alpha3_Ave: {alpha3_Ave}')
print(f'n3_Ave: {n3_Ave}')
print(f'Ks3_Ave: {Ks3_Ave}')
print(f'K0_added_Ave: {K0_added_Ave}')
print(f'L_added_Ave: {L_added_Ave}')

theta_r1_Ave: 0.062
theta_s1_Ave: 0.4
alpha1_Ave: 0.202
n1_Ave: 1.312
Ks1_Ave: 20.205
K0_Ave: 6.588
L_Ave: -1.985
---------------------------
theta_r3_Ave: 0.094
theta_s3_Ave: 0.388
alpha3_Ave: 0.124
n3_Ave: 1.299
Ks3_Ave: 24.986
K0_added_Ave: 1.499
L_added_Ave: -3.815


In [ ]:
### Performing pairwise t-tests between Rosetta1 and Rosetta3 hydraulic parameters and the fitted parameters
from scipy.stats import ttest_ind

t_test = {}
h_parameters = {
    'theta_r': (theta_r1_aligned, theta_r3_aligned),
    'theta_s': (theta_s1_aligned, theta_s3_aligned),
    'alpha': (alpha1_aligned, alpha3_aligned),
    'n': (n1_aligned, n3_aligned),
    'Ks': (Ks1_aligned, Ks3_aligned),
    'K0': (K0_aligned, K0_added_aligned),
    'L': (L_aligned, L_added_aligned)}

for param, (v1, v3) in h_parameters.items():
    _, p_value = ttest_ind(v1, v3, nan_policy='omit')
    t_test[param] = p_value

print("T-test (Rosetta1 vs Rosetta3):")
for param, p_value in t_test.items():
    print(f"{param}: p-value = {p_value:.5f}")

T-test (Rosetta1 vs Rosetta3):
theta_r: p-value = 0.00000
theta_s: p-value = 0.05317
alpha: p-value = 0.00000
n: p-value = 0.42710
Ks: p-value = 0.40227
K0: p-value = 0.00000
L: p-value = 0.00000


In [ ]:
### Performing T-test for the daily average volumetric water contents (Rosetta1 vs Rosetta3)
import scipy.stats as stats

vwc_r1_ave = [value for value in vwc_r1_ave.values() if not np.isnan(value)]
vwc_r3_ave = [value for value in vwc_r3_ave.values() if not np.isnan(value)]

_, p_value = stats.ttest_ind(vwc_r1_ave, vwc_r3_ave)

print(f'P-value: {p_value:.5f}')

P-value: 0.14142


In [ ]:
### Performing T-test for the annual average hydraulic conductivity (Rosetta1 vs Rosetta3)
import scipy.stats as stats

K_r1_ave = [value for value in K_r1_ave.values() if not np.isnan(value)]
K_r3_ave = [value for value in K_r3_ave.values() if not np.isnan(value)]

_, p_value = stats.ttest_ind(K_r1_ave, K_r3_ave)

print(f'P-value: {p_value:.5f}')

P-value: 0.00282


In [ ]:
### Plotting and saving vwc results into a folder

import matplotlib.pyplot as plt

# Create output folder
output_folder = 'vwc_figures_Rosetta1_Rosetta3_comparison'
os.makedirs(output_folder, exist_ok=True)

# Get all unique site IDs
all_sites = sorted(set(vwc_r1.keys()).union(set(vwc_r3.keys())))

# Function to plot data
def plot_vwc_data(ax, time, vwc_data, site_id, label):
  ax.plot(time, vwc_data, label=label)
  ax.set_xlabel('Time')
  ax.set_ylabel('Volumetric water content [cm³ cm⁻³]')
  ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), frameon=False, ncol=2)
  ax.set_title(f'Site: {site_id}', pad=20)
  ax.text(0.02, 0.95, 'a)', transform=ax.transAxes, fontsize=14, verticalalignment='top')

# Plot data for each site
for site_id in all_sites:
  fig, ax = plt.subplots(figsize=(10, 5))

  if site_id in vwc_r1:
    plot_vwc_data(ax, time[site_id], vwc_r1[site_id], site_id, 'Rosetta1')
  if site_id in vwc_r3:
    plot_vwc_data(ax, time[site_id], vwc_r3[site_id], site_id, 'Rosetta3')

  plt.tight_layout()
  fig.savefig(os.path.join(output_folder, f'{site_id}_figure.png'))
  plt.close(fig)

In [ ]:
### Plotting and saving K results into a folder

import matplotlib.pyplot as plt

# Create output folder
output_folder = 'K_figures_Rosetta1_Rosetta3_comparison'
os.makedirs(output_folder, exist_ok=True)

# Get all unique site IDs
all_sites = sorted(set(K_r1.keys()).union(set(K_r3.keys())))

# Function to plot data
def plot_K_data(ax, time, K_data, site_id, label):
  ax.plot(time, K_data, label=label)
  ax.set_xlabel('Time')
  ax.set_ylabel('Hydraulic conductivity [cm d⁻¹]')
  ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), frameon=False, ncol=2)
  ax.set_title(f'Site: {site_id}', pad=20)
  ax.text(0.02, 0.95, 'b)', transform=ax.transAxes, fontsize=14, verticalalignment='top')

# Plot data for each site
for site_id in all_sites:
  fig, ax = plt.subplots(figsize=(10, 5))

  if site_id in K_r1:
    plot_K_data(ax, time[site_id], K_r1[site_id], site_id, 'Rosetta1')
  if site_id in K_r3:
    plot_K_data(ax, time[site_id], K_r3[site_id], site_id, 'Rosetta3')

  plt.tight_layout()
  fig.savefig(os.path.join(output_folder, f'{site_id}_figure.png'))
  plt.close(fig)